Importing required libraries.

In [3]:
import requests
import json
import pandas as pd
import yfinance as yf
!pip install snowflake-connector-python

The first step is to get a complete list of the Fortune 500 companies.
We access an wiki variant api and pull the required data and parse through the json file.

In [29]:
url = "https://www.wikitable2json.com/api/List_of_S%26P_500_companies?table=0"
response = requests.request("GET",url)
list_500 =[]
counter = 1
#Extract the relevant symbology for the Fortune 500 companies
while counter < 501:
  for i in response.json():
    list_500.append(i[counter][0])
  counter += 1
#The particular companies do not have any data
list_500.remove("BF.B")
list_500.remove("BRK.B")

In [109]:
data = yf.download(list_500,period='1mo',interval='1d',start='2024-01-01',end='2024-12-31')

[*********************100%***********************]  498 of 498 completed


In [158]:
data_flat = data.stack(level=1).reset_index()
data_flat.columns = ['Date', 'Ticker', 'Open', 'High', 'Low', 'Close', 'Volume']
data_flat['Date'] = pd.to_datetime(data_flat['Date'])
print(data_flat)

<ipython-input-158-988514380>:1: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  data_flat = data.stack(level=1).reset_index()


             Date Ticker        Open        High         Low       Close      Volume
0      2024-01-02      A  137.501770  139.325213  136.669331  136.946810   1441600.0
1      2024-01-02   AAPL  184.290405  187.070052  182.553128  185.789422  82488700.0
2      2024-01-02   ABBV  151.344757  151.647780  146.562556  146.685652   8059900.0
3      2024-01-02   ABNB  134.479996  135.600006  133.339996  134.429993   3485200.0
4      2024-01-02    ABT  106.644485  107.760929  106.362947  106.362947   5058600.0
...           ...    ...         ...         ...         ...         ...         ...
124743 2024-12-30   WYNN   85.374672   86.936343   84.787798   86.936343   2180100.0
124744 2024-12-30    XEL   65.929565   66.466057   65.510136   66.466057   2642900.0
124745 2024-12-30    XOM  103.865776  104.651443  103.620254  104.396105  11080800.0
124746 2024-12-30    XYL  115.551468  116.068214  114.438490  115.730340    586800.0
124747 2024-12-30    YUM  132.243332  133.124821  131.728300  132

In [43]:
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
from codecs import open

In [10]:

cnn = snowflake.connector.connect(
  account = "VFQPREV-IVC18679",
  user = "CARLOSJZAMORASANTANA",
  password = "Snowflake2025#",
  role = "ACCOUNTADMIN",
  warehouse = "<none selected>",
  database = "<none selected>",
  schema = "<none selected>",
)
print("Connected")

print("Creating Warehouse, Database, and Schema ")
# Creating the table warehosue, database, schema, and table
print("Creating Warhouse...")
cnn.cursor().execute("CREATE WAREHOUSE IF NOT EXISTS financial_warehouse")
print("Done...")
print("Creating Database...")
cnn.cursor().execute("CREATE DATABASE IF NOT EXISTS financial_database")
cnn.cursor().execute("USE DATABASE financial_database")
print("Done...")
print("Creating Schema")
cnn.cursor().execute("CREATE SCHEMA IF NOT EXISTS financial_schema")
cnn.cursor().execute("USE SCHEMA financial_schema")
print("Done")
print("Closing...")
cnn.close()
print("Done..")

Connected
Creating Warehouse, Database, and Schema 
Creating Warhouse...
Done...
Creating Database...
Done...
Creating Schema
Done
Closing...
Done..


In [161]:
# Inserting 2024 financial data into table into Snowflake
# Create a table
table = "financial_table"

# Establishing Connection again but specifying the warehouse, database, etc
print("Establishing Connection...")
cnn = snowflake.connector.connect(
  account = "VFQPREV-IVC18679",
  user = "CARLOSJZAMORASANTANA",
  password = "Snowflake2025#",
  role = "ACCOUNTADMIN",
  warehouse = "financial_warehouse",
  database = "financial_database",
  schema = "financial_schema",
)
print("Connected...")


print("Creating new table to insert DF into ...")
# Insert data frame into table via Snowflake

# # Reset the index, rename the index column to 'DATE', convert other column names to uppercase, and rename the 'ALL' column
# df_to_write = download_data["Close"].reset_index().rename(columns={'index': 'DATE'})
# df_to_write.columns = df_to_write.columns.str.upper()
# # Rename the 'ALL' column after converting to uppercase
# df_to_write.rename(columns={'ALL': 'ALL_TICKER'}, inplace=True)


write_pandas(cnn, data_flat, table, auto_create_table=True,use_deprecated_int96_timestamps=True, allow_truncated_timestamps=True)
print('Completed...')
print("Checking execution...")
print("Closing Connection...")
cnn.close()
print('Done')

Establishing Connection...
Connected...
Creating new table to insert DF into ...
Completed...
Checking execution...
Closing Connection...
Done


In [186]:
#Get the industry and sector for each company
#Will make creating a dashboard easier
columns_name = ["Company", "Industry", "Sector"]
df = pd.DataFrame(columns=columns_name)
for i in list_500:
  df.loc[len(df)] = [i,yf.Ticker(i).get_info()["industry"],yf.Ticker(i).get_info()["sector"]]

In [187]:
print(df)

    Company                         Industry             Sector
0       MMM                    Conglomerates        Industrials
1       AOS   Specialty Industrial Machinery        Industrials
2       ABT                  Medical Devices         Healthcare
3      ABBV     Drug Manufacturers - General         Healthcare
4       ACN  Information Technology Services         Technology
..      ...                              ...                ...
493    WDAY           Software - Application         Technology
494    WYNN                Resorts & Casinos  Consumer Cyclical
495     XEL   Utilities - Regulated Electric          Utilities
496     XYL   Specialty Industrial Machinery        Industrials
497     YUM                      Restaurants  Consumer Cyclical

[498 rows x 3 columns]


In [188]:
#Inserting another table in snowflake
table = "industry_sector_table"
#Establishing Connections
print("Establishing Connection...")
cnn = snowflake.connector.connect(
  account = "VFQPREV-IVC18679",
  user = "CARLOSJZAMORASANTANA",
  password = "Snowflake2025#",
  role = "ACCOUNTADMIN",
  warehouse = "financial_warehouse",
  database = "financial_database",
  schema = "financial_schema",
)
print("Connected...")
print("Creating new table to insert DF into ...")

write_pandas(cnn, df, table, auto_create_table=True,use_deprecated_int96_timestamps=True, allow_truncated_timestamps=True)
print('Completed...')
print("Checking execution...")
print("Closing Connection...")
cnn.close()
print('Done')

Establishing Connection...
Connected...
Creating new table to insert DF into ...


<ipython-input-188-1743240711>:17: UserWarning: Pandas Dataframe has non-standard index of type <class 'pandas.core.indexes.base.Index'> which will not be written. Consider changing the index to pd.RangeIndex(start=0,...,step=1) or call reset_index() to keep index as column(s)
  write_pandas(cnn, df, table, auto_create_table=True,use_deprecated_int96_timestamps=True, allow_truncated_timestamps=True)


Completed...
Checking execution...
Closing Connection...
Done
